# All filters generator

### Parameters

In [29]:
#################################
# IMPORTANT SETUP
WRITE_ALL_FILTERS_FILE = True          # Overwrite the filters modules t%_affine_%.v
WRITE_ALL_FILTERS_TOPO_FILE = True     # Overwrite the all_filters_%.v file
MODIFIED = True                         # Change modified date
#################################


FRAC = 15                               # Number of fractional pixels for interpolation (eg: 1/4 of resolution)
TAPS = 6                                # Number of taps in the filter
PADDING = 2                             # How much padding to add in each four sides
ROUNDING_TERM = 64                      # Number used to divide the sums (= sum of all coefficients in horizontal)
IN_NUM = 6                              # 1 line of 6 samples (6-tap)
IN_BIT_WIDTH = 11                        # Number of bits per sample
OUT_NUM = 15                            # There are 15 filters used to generate 1/16 precision


#### Coefficients

In [30]:
coefficients = [
                    [ 1,  -3,  63,   4,  -2, 1],
                    [ 1,  -5,  62,   8,  -3, 1],
                    [ 2,  -8,  60,  13,  -4, 1],

                    [ 3, -10,  58,  17,  -5, 1],
                    [ 3, -11,  52,  26,  -8, 2],
                    [ 2,  -9,  47,  31, -10, 3],
                    [ 3, -11,  45,  34, -10, 3],

                    [ 3, -11,  40,  40, -11, 3],
                    [ 3, -10,  34,  45, -11, 3],
                    [ 3, -10,  31,  47,  -9, 2],
                    [ 2,  -8,  26,  52, -11, 3],

                    [ 1,  -5,  17,  58, -10, 3],
                    [ 1,  -4,  13,  60,  -8, 2],
                    [ 1,  -3,   8,  62,  -5, 1],
                    [ 1,  -2,   4,  63,  -3, 1]
                    ]

In [31]:
# Pre calculated in Google Sheets [TCC] Affine interpolation of a 4x4
# coefs_sizes = [
#                 [8,	10,	14, 10,	10,	8],
#                 [8,	11,	14,	11,	10,	8],
#                 [9,	12,	14,	12,	11,	8],
#                 [10, 12, 14, 13, 11,8],
#                 [10, 12, 14, 13, 12,9],
#                 [9,	12, 14,	13,	12,	10],
#                 [10, 12, 14, 14, 12, 10],
#                 [10, 12, 14, 14, 12, 10],
#                 [10, 12, 14, 14, 12, 10],
#                 [10, 12, 13, 14, 12, 9],
#                 [9,	12,	13,	14,	12,	10],
#                 [8,	11,	13,	14,	12,	10],
#                 [8,	11,	12,	14,	12,	9],
#                 [8,	10,	11,	14,	11,	8],
#                 [8,	10,	10,	14,	10,	8]]

In [32]:
import math
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime 

### Variables calculation

#### Input and outputs

In [33]:
# Input size
in_width = IN_NUM * IN_BIT_WIDTH

# Input text
in_text = "\tinput signed ["+str(in_width-1)+":0] X;"

# Output ports
out_names = []
for i in range(1,OUT_NUM+1):
    if i != OUT_NUM:
        out_names.append('\tY'+str(i)+',')
    else:
        out_names.append('\tY'+str(i))
out_names_concat = '\n'.join(out_names)

# print(in_text)
# print(out_names_concat)

Obs: outputs sizes are calculated after all calculations are ready in the end of this code

In [34]:
coefs_sizes = [[0 for x in range(TAPS)] for y in range(FRAC)] 
max_neg_input = -(2**(IN_BIT_WIDTH-1))
max_pos_input = 2**(IN_BIT_WIDTH-1)-1

for j in range(0,TAPS):
    for i in range(0,FRAC):
        coef = coefficients[i][j]
        bit_width_pos = max_pos_input*coef
        bit_width_neg = max_neg_input*coef
        if bit_width_pos >= 0:      
            if bit_width_pos <= 2**(IN_BIT_WIDTH-1)-1 and bit_width_neg >= -2**((IN_BIT_WIDTH-1)):
                coefs_sizes[i][j] = IN_BIT_WIDTH
            elif bit_width_pos <= 2**(IN_BIT_WIDTH)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH):
                coefs_sizes[i][j] = IN_BIT_WIDTH+1
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+1)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+1):
                coefs_sizes[i][j] = IN_BIT_WIDTH+2
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+2)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+2):
                coefs_sizes[i][j] = IN_BIT_WIDTH+3
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+3)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+3):
                coefs_sizes[i][j] = IN_BIT_WIDTH+4
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+4)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+4):
                coefs_sizes[i][j] = IN_BIT_WIDTH+5
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+5)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+5):
                coefs_sizes[i][j] = IN_BIT_WIDTH+6
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+6)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+6):
                coefs_sizes[i][j] = IN_BIT_WIDTH+7
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+7)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+7):
                coefs_sizes[i][j] = IN_BIT_WIDTH+8
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+8)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+8):
                coefs_sizes[i][j] = IN_BIT_WIDTH+9
            elif bit_width_pos <= 2**(IN_BIT_WIDTH+9)-1 and bit_width_neg >= -2**(IN_BIT_WIDTH+9):
                coefs_sizes[i][j] = IN_BIT_WIDTH+10
            else:
                print('#############################')
        else:
            if bit_width_neg <= 2**(IN_BIT_WIDTH-1)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH-1):
                coefs_sizes[i][j] = IN_BIT_WIDTH
            elif bit_width_neg <= 2**(IN_BIT_WIDTH)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH):
                coefs_sizes[i][j] = IN_BIT_WIDTH+1
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+1)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+1):
                coefs_sizes[i][j] = IN_BIT_WIDTH+2
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+2)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+2):
                coefs_sizes[i][j] = IN_BIT_WIDTH+3
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+3)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+3):
                coefs_sizes[i][j] = IN_BIT_WIDTH+4
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+4)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+4):
                coefs_sizes[i][j] = IN_BIT_WIDTH+5
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+5)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+5):
                coefs_sizes[i][j] = IN_BIT_WIDTH+6
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+6)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+6):
                coefs_sizes[i][j] = IN_BIT_WIDTH+7
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+7)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+7):
                coefs_sizes[i][j] = IN_BIT_WIDTH+8
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+8)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+8):
                coefs_sizes[i][j] = IN_BIT_WIDTH+9
            elif bit_width_neg <= 2**(IN_BIT_WIDTH+9)-1 and bit_width_pos >= -2**(IN_BIT_WIDTH+9):
                coefs_sizes[i][j] = IN_BIT_WIDTH+10
            else:
                print('#############################')

# coefs_sizes

#### Filter outputs names and module instantiation

In [35]:
# To instantiate module: .Y1 (r_t0_f1_1);
filters_out = []

# To create wire: wire signed [7:0] r_t0_f1_1;
filters_wire = []

# Stores filter's output wires in the same shape of the coefficients
wire_out_filter_name = [[0 for x in range(TAPS)] for y in range(FRAC)] 

for t in range(0,TAPS):
    filters_wire.append('\n\t// TAP '+str(t))

    filters_out.append('\n\t t'+str(t)+'_affine_'+str(IN_BIT_WIDTH)+' T'+str(t)+'(')
    filters_out.append('\t\t.X  (X'+str(t)+'),')
    for y in range(1,FRAC+1):
        c = coefficients[y-1][t]
        s = coefs_sizes[y-1][t]
        if y != 15:
            if c > 0:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(c)
            else:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg'
            filters_out.append('\t\t.Y'+str(y)+'\t('+str(wire_out_filter_name[y-1][t])+'),')
        else:
            if c > 0:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(c)
            else:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg'
            filters_out.append('\t\t.Y'+str(y)+'\t('+str(wire_out_filter_name[y-1][t])+')')
        filters_wire.append('\twire signed ['+str(s-1)+':0]\t'+str(wire_out_filter_name[y-1][t])+';')
    filters_out.append('\t);\n')

# To instantiate module: .Y1 (r_t0_f1_1);
filters_out_concat = '\n'.join(filters_out)

# To create wire: wire signed [7:0] r_t0_f1_1
filters_wire_concat = '\n'.join(filters_wire)

# print(filters_wire_concat)
print(filters_out_concat)


	 t0_affine_11 T0(
		.X  (X0),
		.Y1	(r_t0_f1_1),
		.Y2	(r_t0_f2_1),
		.Y3	(r_t0_f3_2),
		.Y4	(r_t0_f4_3),
		.Y5	(r_t0_f5_3),
		.Y6	(r_t0_f6_2),
		.Y7	(r_t0_f7_3),
		.Y8	(r_t0_f8_3),
		.Y9	(r_t0_f9_3),
		.Y10	(r_t0_f10_3),
		.Y11	(r_t0_f11_2),
		.Y12	(r_t0_f12_1),
		.Y13	(r_t0_f13_1),
		.Y14	(r_t0_f14_1),
		.Y15	(r_t0_f15_1)
	);


	 t1_affine_11 T1(
		.X  (X1),
		.Y1	(r_t1_f1_3neg),
		.Y2	(r_t1_f2_5neg),
		.Y3	(r_t1_f3_8neg),
		.Y4	(r_t1_f4_10neg),
		.Y5	(r_t1_f5_11neg),
		.Y6	(r_t1_f6_9neg),
		.Y7	(r_t1_f7_11neg),
		.Y8	(r_t1_f8_11neg),
		.Y9	(r_t1_f9_10neg),
		.Y10	(r_t1_f10_10neg),
		.Y11	(r_t1_f11_8neg),
		.Y12	(r_t1_f12_5neg),
		.Y13	(r_t1_f13_4neg),
		.Y14	(r_t1_f14_3neg),
		.Y15	(r_t1_f15_2neg)
	);


	 t2_affine_11 T2(
		.X  (X2),
		.Y1	(r_t2_f1_63),
		.Y2	(r_t2_f2_62),
		.Y3	(r_t2_f3_60),
		.Y4	(r_t2_f4_58),
		.Y5	(r_t2_f5_52),
		.Y6	(r_t2_f6_47),
		.Y7	(r_t2_f7_45),
		.Y8	(r_t2_f8_40),
		.Y9	(r_t2_f9_34),
		.Y10	(r_t2_f10_31),
		.Y11	(r_t2_f11_26),
		.Y12	(r_t2_f12_17),
		.Y1

#### Dividing the input into 6 smaller parts

In [36]:
# Declaration of the wire's names used to split the input 
wire_in_split_declaration = []

wire_in_split_declaration.append('wire signed ['+str(IN_BIT_WIDTH-1)+':0]')
for n in range(IN_NUM):
    if n < IN_NUM-1:
        wire_in_split_declaration.append('X'+str(n)+',')
    else:
        wire_in_split_declaration.append('X'+str(n)+';')

wire_in_split_declaration_concat = ' '.join(wire_in_split_declaration)

# Assigning the input wires 
wire_in_split_calculation = []
MSB_in_split = IN_BIT_WIDTH*TAPS - 1
LSB_in_split = IN_BIT_WIDTH*TAPS - IN_BIT_WIDTH

for n in range(IN_NUM):
    wire_in_split_calculation.append('\tassign X'+str(n)+' = X['+str(MSB_in_split)+':'+str(LSB_in_split)+'];')
    MSB_in_split = MSB_in_split - IN_BIT_WIDTH
    LSB_in_split = LSB_in_split - IN_BIT_WIDTH
wire_in_split_calculation_concat = '\n'.join(wire_in_split_calculation)

# print(wire_in_split_declaration_concat)
# print(wire_in_split_calculation_concat)

#### First round sums

In [37]:
# wire for first sums [15][3]: s_f1_t0_t1
wire_sum_1_name = [[0 for x in range(int(TAPS/2))] for y in range(FRAC)] 
# print(len(wire_sum_1_name),len(wire_sum_1_name[0]))
# Sizes of the resulting first sums (size for wire_sum_1_name)
wire_sum_1_size = [[0 for x in range(int(TAPS/2))] for y in range(FRAC)]

# wire declaration for first sums
wire_sum_1_declaration = []

# First sums calculation
wire_sum_1_calculation = []

for f in range(1,FRAC+1):
    for t in range(0,int(TAPS/2)):
        wire_sum_1_name[f-1][t] = 's_f'+str(f)+'_t'+str(2*t)+'_t'+str(2*t+1)

        s1 = coefs_sizes[f-1][2*t]                  # Size of p1
        s2 = coefs_sizes[f-1][2*t+1]                # Size of p2
        
        name1 = wire_out_filter_name[f-1][2*t]
        name2 = wire_out_filter_name[f-1][2*t+1]
        # print(s1,s2)

        if s1 > s2:
            p1 = '\t{'+str(name1)+'['+str(s1-1)+'], '+str(name1)+'}'
            p2 = ' + {'

            for bit in range(0,s1-s2+1):
                p2 = p2+str(name2)+'['+str(s2-1)+'], '
                
            p2 = p2+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s1+1)+' bits'
            wire_sum_1_size[f-1][t] = s1 + 1
        else:
            p1 = '\t{'
            p2 = ' + {'+str(name2)+'['+str(s2-1)+'], '+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s2+1)+' bits'

            for bit in range(0,s2-s1+1):
                p1 = p1 + str(name1)+'['+str(s1-1)+'], '

            p1 = p1+str(name1)+'}'
            wire_sum_1_size[f-1][t] = s2 + 1

        wire_sum_1_declaration.append('\twire signed ['+str(wire_sum_1_size[f-1][t]-1)+':0]\t'+str(wire_sum_1_name[f-1][t])+';\t// '+str(wire_sum_1_size[f-1][t])+' bits')
        wire_sum_1_calculation.append('\t\tassign '+str(wire_sum_1_name[f-1][t])+' = '+str(p1)+str(p2))

wire_sum_1_declaration_concat = '\n'.join(wire_sum_1_declaration)
wire_sum_1_calculation_concat = '\n'.join(wire_sum_1_calculation)

# print(wire_sum_1_declaration_concat)
# print(wire_sum_1_calculation_concat)

#### Second round sums

Essa parte só funciona para filtros de 6 taps

In [38]:
# wire for second sums [15][3]: s_f1_0
wire_sum_2_name = [[0 for x in range(2)] for y in range(FRAC)] 

# Sizes of the resulting first sums (size for wire_sum_1_name)
wire_sum_2_size = [[0 for x in range(2)] for y in range(FRAC)]

# wire declaration for second sums
wire_sum_2_declaration = []

# Second sums calculation
wire_sum_2_calculation = []

for f in range(1,FRAC+1):
    for t in range(0,2):
        wire_sum_2_name[f-1][t] = 's_f' + str(f) + '_' + str(t)

        s1 = wire_sum_1_size[f-1][2*t]   
        name1 = wire_sum_1_name[f-1][2*t]

        if t == 1:
            s2 = wire_sum_2_size[f-1][t-1]
            name2 = wire_sum_2_name[f-1][t-1]          
        else:
            s2 = wire_sum_1_size[f-1][2*t+1]   
            name2 = wire_sum_1_name[f-1][2*t+1]

        if s1 > s2:
            p1 = '\t{'+str(name1)+'['+str(s1-1)+'], '+str(name1)+'}'
            p2 = ' + {'
            
            for bit in range(0,s1-s2+1):
                p2 = p2+str(name2)+'['+str(s2-1)+'], ' 
            p2 = p2+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s1+1)+' bits'
            
            wire_sum_1_size[f-1][t] = s1 + 1
        else:
            p1 = '\t{'
            p2 = ' + {'+str(name2)+'['+str(s2-1)+'], '+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s2+1)+' bits'

            for bit in range(0,s2-s1+1):
                p1 = p1 + str(name1)+'['+str(s1-1)+'],  '
            p1 = p1+str(name1)+'}'

            wire_sum_2_size[f-1][t] = s2 + 1

        wire_sum_2_declaration.append('\twire signed ['+str(wire_sum_2_size[f-1][t]-1)+':0]\t'+str(wire_sum_2_name[f-1][t])+';\t// '+str(wire_sum_2_size[f-1][t])+' bits')
        wire_sum_2_calculation.append('\t\tassign '+str(wire_sum_2_name[f-1][t])+' = '+str(p1)+str(p2))

wire_sum_2_declaration_concat = '\n'.join(wire_sum_2_declaration)
wire_sum_2_calculation_concat = '\n'.join(wire_sum_2_calculation)

# print(wire_sum_2_declaration_concat)
# print(wire_sum_2_calculation_concat)

#### Applying the rounding term 

In [39]:
# To store all shifts left of the final sum
wire_sum_shr_name = []

# To store the sizes of wire_sum_shr_name
wire_sum_shr_size = []

# To write the wire declaration
shr_declaration = []

# To write the calculations
shr_calculation = []

# Shifts to outputs
shr_to_out = []

# Number of left shifts
shifts = int(math.log(ROUNDING_TERM,2))

for f in range(1,FRAC+1):
    wire_sum_shr_name.append('s_f'+str(f)+'_shr')
    wire_sum_shr_size.append(int(wire_sum_2_size[f-1][1]-shifts))

    shr_declaration.append('\twire signed ['+str(wire_sum_shr_size[f-1]-1)+':0]\t'+str(wire_sum_shr_name[f-1])+';\t // '+str(wire_sum_shr_size[f-1])+' bits')
    shr_calculation.append('\t\tassign '+str(wire_sum_shr_name[f-1])+' = '+str(wire_sum_2_name[f-1][1])+' >> '+str(shifts)+';')
    shr_to_out.append('\tassign Y'+str(f)+' = '+str(wire_sum_shr_name[f-1])+';\t// '+str(wire_sum_shr_size[f-1])+' bits')

shr_declaration_concat = '\n'.join(shr_declaration)
shr_calculation_concat = '\n'.join(shr_calculation)
shr_to_out_concat = '\n'.join(shr_to_out)

# print(shr_declaration_concat)
# print(shr_calculation_concat)
# print(shr_to_out_concat)

### Ouput declaration (after all calculations are ready)

In [40]:
# Output port modes
out_port = []
for i in range(1,OUT_NUM+1):
    out_port.append('\toutput signed ['+str(wire_sum_shr_size[i-1]-1)+':0] Y'+str(i)+';')
out_port_concat = '\n'.join(out_port)

# print(out_port_concat)

### Final Verilog text

In [41]:
if MODIFIED:
    DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

In [42]:
all_filters_verilog = '''/*------------------------------------------------------------------------------
 * File: all_filters_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 05/12/2022
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters in a sum tree
 *------------------------------------------------------------------------------ */

module all_filters_'''+str(IN_BIT_WIDTH)+''' (
    X,
'''+out_names_concat+'''
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------

'''+in_text+'''
'''+out_port_concat+'''

// ------------------------------------------
// Wires declarations
// ------------------------------------------
	
	// Spliting the input into parts of '''+str(IN_BIT_WIDTH)+''' bits each
	'''+wire_in_split_declaration_concat+'''
	
	// Regs for filter's outputs 
	'''+filters_wire_concat+'''
	
	// First sums
'''+wire_sum_1_declaration_concat+'''
		
	// Second sums
'''+wire_sum_2_declaration_concat+'''
	
	// Final sums shifted right
'''+shr_declaration_concat+'''
 
	 
// ------------------------------------------
// Modules instantiation
// ------------------------------------------
'''+filters_out_concat+'''

// ------------------------------------------
// Combinational logic
// ------------------------------------------

	// Splitting input into smaller parts
'''+wire_in_split_calculation_concat+'''
  
	// Procedure for sums	
		// First sums
'''+wire_sum_1_calculation_concat+'''
		
		// Second sums
'''+wire_sum_2_calculation_concat+'''

		// Shifting for division by 64
'''+shr_calculation_concat+'''

// ------------------------------------------
// Outputs
// ------------------------------------------

'''+shr_to_out_concat+'''


endmodule // all_filters_'''+str(IN_BIT_WIDTH)+'''


'''
if WRITE_ALL_FILTERS_TOPO_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/all_filters_'+str(IN_BIT_WIDTH)+'.v', 'w') as all_filters_file:
        all_filters_file.write(all_filters_verilog)
        
# print(all_filters_verilog)  

## Generating all filter modules 

#### T0

In [43]:
# Wires declarations for T0
wires_coefs_t0 = [1,4,3,2]

# Calculating wires sizes based on wires_coefs_t0
wires_sizes_t0 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+2,IN_BIT_WIDTH+1]

# Display error message
if len(wires_coefs_t0) != len(wires_sizes_t0):
    print('#### ERROR: wires_coefs_t0 and wires_sizes_t0 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t0 = []
for w in range(len(wires_coefs_t0)):
    if wires_coefs_t0[w] >= 0:
        wires_declarations_t0.append('  wire signed ['+str(wires_sizes_t0[w]-1)+':0] w'+str(wires_coefs_t0[w])+';')
    else:
        wires_declarations_t0.append('  wire signed ['+str(wires_sizes_t0[w]-1)+':0] w'+str(abs(wires_coefs_t0[w]))+'_;')

wires_declarations_t0_concat = '\n'.join(wires_declarations_t0)

# Creates the outputs sizes for declaration
output_declarations_t0 = []
for i in range(FRAC):
    output_declarations_t0.append('  output signed ['+str(coefs_sizes[i][0]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t0_concat = '\n'.join(output_declarations_t0)

# print(output_declarations_t0_concat)
# print(wires_declarations_t0_concat)

In [ ]:
t0_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t0_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 0
 *------------------------------------------------------------------------------ */

module t0_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed  ['''+str(IN_BIT_WIDTH-1)+''':0] 	X;
'''+output_declarations_t0_concat+'''
	 

// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t0_concat+'''


// ------------------------------------------
// Combinational logic
// ------------------------------------------

  assign w1 = X;
  assign w4 = w1 << 2;
  assign w3 = w4 - w1;
  assign w2 = w1 << 1;
  
  
// ------------------------------------------
// Outputs
// ------------------------------------------

  assign Y1 = w1;
  assign Y2 = w1;
  assign Y3 = w2;
  assign Y4 = w3;
  assign Y5 = w3;
  assign Y6 = w2;
  assign Y7 = w3;
  assign Y8 = w3;
  assign Y9 = w3;
  assign Y10 = w3;
  assign Y11 = w2;
  assign Y12 = w1;
  assign Y13 = w1;
  assign Y14 = w1;
  assign Y15 = w1;

endmodule // t0_affine_'''+str(IN_BIT_WIDTH)+'''

'''

# print(t0_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t0_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t0_affine_file:
        t0_affine_file.write(t0_affine_verilog)

#### T1

In [ ]:
# Wires declarations for T0
wires_coefs_t1 = [1,4,3,5,8,9,11,-3,-5,-8,10,-10,-11,-9,-4,2,-2]

# Calculating wires sizes based on wires_coefs_t0
wires_sizes_t1 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+2,IN_BIT_WIDTH+3,IN_BIT_WIDTH+3,
                  IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+2,IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,
                  IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+3,
                  IN_BIT_WIDTH+1,IN_BIT_WIDTH+2
                  ]

# Display error message
if len(wires_coefs_t1) != len(wires_sizes_t1):
    print('#### ERROR: wires_coefs_t1 and wires_sizes_t1 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t1 = []
for w in range(len(wires_coefs_t1)):
    if wires_coefs_t1[w] >= 0:
        wires_declarations_t1.append('  wire signed ['+str(wires_sizes_t1[w]-1)+':0] w'+str(wires_coefs_t1[w])+';')
    else:
        wires_declarations_t1.append('  wire signed ['+str(wires_sizes_t1[w]-1)+':0] w'+str(abs(wires_coefs_t1[w]))+'_;')

wires_declarations_t1_concat = '\n'.join(wires_declarations_t1)

# Creates the outputs sizes for declaration
output_declarations_t1 = []
for i in range(FRAC):
    output_declarations_t1.append('  output signed ['+str(coefs_sizes[i][1]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t1_concat = '\n'.join(output_declarations_t1)

# print(output_declarations_t1_concat)
# print(wires_declarations_t1_concat)

In [ ]:
t1_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t1_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 1
 *------------------------------------------------------------------------------ */

module t1_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed  ['''+str(IN_BIT_WIDTH-1)+''':0] X;
'''+output_declarations_t1_concat+'''
	 

// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t1_concat+'''


// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign w1 = X;
  assign w4 = w1 << 2;
  assign w3 = w4 - w1;
  assign w5 = w1 + w4;
  assign w8 = w1 << 3;
  assign w9 = w1 + w8;
  assign w11 = w3 + w8;
  assign w3_ = -1 * w3;
  assign w5_ = -1 * w5;
  assign w8_ = -1 * w8;
  assign w10 = w5 << 1;
  assign w10_ = -1 * w10;
  assign w11_ = -1 * w11;
  assign w9_ = -1 * w9;
  assign w4_ = -1 * w4;
  assign w2 = w1 << 1;
  assign w2_ = -1 * w2;
  
  
// ------------------------------------------
// Outputs
// ------------------------------------------
  assign Y1  = w3_;
  assign Y2  = w5_;
  assign Y3  = w8_;
  assign Y4  = w10_;
  assign Y5  = w11_;
  assign Y6  = w9_;
  assign Y7  = w11_;
  assign Y8  = w11_;
  assign Y9  = w10_;
  assign Y10  = w10_;
  assign Y11 = w8_;
  assign Y12 = w5_;
  assign Y13 = w4_;
  assign Y14 = w3_;
  assign Y15 = w2_;

endmodule //t1_affine_'''+str(IN_BIT_WIDTH)+'''

'''

# print(t1_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t1_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t1_affine_file:
        t1_affine_file.write(t1_affine_verilog)

#### T2

In [ ]:
# Wires declarations for T2
wires_coefs_t2 = [1,4,5,16,15,17,32,31,64,63,8,13,30,29,40,45,47,62,60,58,52,34,26]

# Calculating wires sizes based on wires_coefs_t2
wires_sizes_t2 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,
                  IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+5
                ]

# Display error message
if len(wires_coefs_t2) != len(wires_sizes_t2):
    print('#### ERROR: wires_coefs_t2 and wires_sizes_t2 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t2 = []
for w in range(len(wires_coefs_t2)):
    if wires_coefs_t2[w] >= 0:
        wires_declarations_t2.append('  wire signed ['+str(wires_sizes_t2[w]-1)+':0] w'+str(wires_coefs_t2[w])+';')
    else:
        wires_declarations_t2.append('  wire signed ['+str(wires_sizes_t2[w]-1)+':0] w'+str(abs(wires_coefs_t2[w]))+'_;')

wires_declarations_t2_concat = '\n'.join(wires_declarations_t2)

# Creates the outputs sizes for declaration
output_declarations_t2 = []
for i in range(FRAC):
    output_declarations_t2.append('  output signed ['+str(coefs_sizes[i][2]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t2_concat = '\n'.join(output_declarations_t2)

# print(output_declarations_t2_concat)
# print(wires_declarations_t2_concat)

In [ ]:
t2_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t2_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 2
 *------------------------------------------------------------------------------ */

module t2_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed ['''+str(IN_BIT_WIDTH-1)+''':0] X;
'''+output_declarations_t2_concat+'''


// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t2_concat+'''
	 
	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign w1 = X;
  assign w4 = w1 << 2;
  assign w5 = w1 + w4;
  assign w16 = w1 << 4;
  assign w15 = w16 - w1;
  assign w17 = w1 + w16;
  assign w32 = w1 << 5;
  assign w31 = w32 - w1;
  assign w64 = w1 << 6;
  assign w63 = w64 - w1;
  assign w8 = w1 << 3;
  assign w13 = w5 + w8;
  assign w30 = w15 << 1;
  assign w29 = w30 - w1;
  assign w40 = w5 << 3;
  assign w45 = w5 + w40;
  assign w47 = w15 + w32;
  assign w62 = w31 << 1;
  assign w60 = w15 << 2;
  assign w58 = w29 << 1;
  assign w52 = w13 << 2;
  assign w34 = w17 << 1;
  assign w26 = w13 << 1;
  
  
// ------------------------------------------
// Outputs
// ------------------------------------------
  assign Y1 = w63;
  assign Y2 = w62;
  assign Y3 = w60;
  assign Y4 = w58;
  assign Y5 = w52;
  assign Y6 = w47;
  assign Y7 = w45;
  assign Y8 = w40;
  assign Y9 = w34;
  assign Y10 = w31;
  assign Y11 = w26;
  assign Y12 = w17;
  assign Y13 = w13;
  assign Y14 = w8;
  assign Y15 = w4;

endmodule //t2_affine_'''+str(IN_BIT_WIDTH)+'''

'''

# print(t2_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t2_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t2_affine_file:
        t2_affine_file.write(t2_affine_verilog)

#### T3

In [ ]:
# Wires declarations for T3
wires_coefs_t3 = [1,4,5,16,15,17,32,31,64,63,8,13,30,29,40,45,47,26,34,52,58,60,62]

# Calculating wires sizes based on wires_coefs_t3
wires_sizes_t3 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,
                  IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,IN_BIT_WIDTH+5,IN_BIT_WIDTH+5,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+5,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,
                  IN_BIT_WIDTH+6,IN_BIT_WIDTH+6,IN_BIT_WIDTH+6
                ]

# Display error message
if len(wires_coefs_t3) != len(wires_sizes_t3):
    print('#### ERROR: wires_coefs_t3 and wires_sizes_t3 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t3 = []
for w in range(len(wires_coefs_t3)):
    if wires_coefs_t3[w] >= 0:
        wires_declarations_t3.append('  wire signed ['+str(wires_sizes_t3[w]-1)+':0] w'+str(wires_coefs_t3[w])+';')
    else:
        wires_declarations_t3.append('  wire signed ['+str(wires_sizes_t3[w]-1)+':0] w'+str(abs(wires_coefs_t3[w]))+'_;')

wires_declarations_t3_concat = '\n'.join(wires_declarations_t3)

# Creates the outputs sizes for declaration
output_declarations_t3 = []
for i in range(FRAC):
    output_declarations_t3.append('  output signed ['+str(coefs_sizes[i][3]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t3_concat = '\n'.join(output_declarations_t3)

# print(output_declarations_t3_concat)
# print(wires_declarations_t3_concat)

In [ ]:
t3_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t3_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 3
 *------------------------------------------------------------------------------ */

module t3_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed  ['''+str(IN_BIT_WIDTH-1)+''':0] X;
'''+output_declarations_t3_concat+'''


// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t3_concat+'''
	 

// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign w1 = X;
  assign w4 = w1 << 2;
  assign w5 = w1 + w4;
  assign w16 = w1 << 4;
  assign w15 = w16 - w1;
  assign w17 = w1 + w16;
  assign w32 = w1 << 5;
  assign w31 = w32 - w1;
  assign w64 = w1 << 6;
  assign w63 = w64 - w1;
  assign w8 = w1 << 3;
  assign w13 = w5 + w8;
  assign w30 = w15 << 1;
  assign w29 = w30 - w1;
  assign w40 = w5 << 3;
  assign w45 = w5 + w40;
  assign w47 = w15 + w32;
  assign w26 = w13 << 1;
  assign w34 = w17 << 1;
  assign w52 = w13 << 2;
  assign w58 = w29 << 1;
  assign w60 = w15 << 2;
  assign w62 = w31 << 1;
  
  
// ------------------------------------------
// Outputs
// ------------------------------------------
  assign Y1 = w4;
  assign Y2 = w8;
  assign Y3 = w13;
  assign Y4 = w17;
  assign Y5 = w26;
  assign Y6 = w31;
  assign Y7 = w34;
  assign Y8 = w40;
  assign Y9 = w45;
  assign Y10 = w47;
  assign Y11 = w52;
  assign Y12 = w58;
  assign Y13 = w60;
  assign Y14 = w62;
  assign Y15 = w63;

endmodule //t3_affine_'''+str(IN_BIT_WIDTH)+'''

'''
# print(t3_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t3_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t3_affine_file:
        t3_affine_file.write(t3_affine_verilog)

#### T4

In [ ]:
# Wires declarations for T4
wires_coefs_t4 = [1,4,3,5,8,9,11,2,-2,-3,-4,-5,-8,10,-10,-11,-9]

# Calculating wires sizes based on wires_coefs_t4
wires_sizes_t4 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+2,IN_BIT_WIDTH+3,IN_BIT_WIDTH+3,
                  IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+1,IN_BIT_WIDTH+2,IN_BIT_WIDTH+2,
                  IN_BIT_WIDTH+3,IN_BIT_WIDTH+3,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,IN_BIT_WIDTH+4,
                  IN_BIT_WIDTH+4,IN_BIT_WIDTH+4
                ]

# Display error message
if len(wires_coefs_t4) != len(wires_sizes_t4):
    print('#### ERROR: wires_coefs_t4 and wires_sizes_t4 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t4 = []
for w in range(len(wires_coefs_t4)):
    if wires_coefs_t4[w] >= 0:
        wires_declarations_t4.append('  wire signed ['+str(wires_sizes_t4[w]-1)+':0] w'+str(wires_coefs_t4[w])+';')
    else:
        wires_declarations_t4.append('  wire signed ['+str(wires_sizes_t4[w]-1)+':0] w'+str(abs(wires_coefs_t4[w]))+'_;')

wires_declarations_t4_concat = '\n'.join(wires_declarations_t4)

# Creates the outputs sizes for declaration
output_declarations_t4 = []
for i in range(FRAC):
    output_declarations_t4.append('  output signed ['+str(coefs_sizes[i][4]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t4_concat = '\n'.join(output_declarations_t4)

# print(output_declarations_t4_concat)
# print(wires_declarations_t4_concat)

In [ ]:
t4_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t4_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 4
 *------------------------------------------------------------------------------ */

module t4_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed ['''+str(IN_BIT_WIDTH-1)+''':0] X;
'''+output_declarations_t4_concat+'''


// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t4_concat+'''
	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign w1 = X;
  assign w4 = w1 << 2;
  assign w3 = w4 - w1;
  assign w5 = w1 + w4;
  assign w8 = w1 << 3;
  assign w9 = w1 + w8;
  assign w11 = w3 + w8;
  assign w2 = w1 << 1;
  assign w2_ = -1 * w2;
  assign w3_ = -1 * w3;
  assign w4_ = -1 * w4;
  assign w5_ = -1 * w5;
  assign w8_ = -1 * w8;
  assign w10 = w5 << 1;
  assign w10_ = -1 * w10;
  assign w11_ = -1 * w11;
  assign w9_ = -1 * w9;

  
// ------------------------------------------
// Outputs
// ------------------------------------------
  assign Y1 = w2_;
  assign Y2 = w3_;
  assign Y3 = w4_;
  assign Y4 = w5_;
  assign Y5 = w8_;
  assign Y6 = w10_;
  assign Y7 = w10_;
  assign Y8 = w11_;
  assign Y9 = w11_;
  assign Y10 = w9_;
  assign Y11 = w11_;
  assign Y12 = w10_;
  assign Y13 = w8_;
  assign Y14 = w5_;
  assign Y15 = w3_;

endmodule //t4_affine_'''+str(IN_BIT_WIDTH)+'''

'''
# print(t4_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t4_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t4_affine_file:
        t4_affine_file.write(t4_affine_verilog)

#### T5


In [ ]:
# Wires declarations for T5
wires_coefs_t5 = [1,4,3,2]

# Calculating wires sizes based on wires_coefs_t5
wires_sizes_t5 = [IN_BIT_WIDTH,IN_BIT_WIDTH+2,IN_BIT_WIDTH+2,IN_BIT_WIDTH+1]

# Display error message
if len(wires_coefs_t5) != len(wires_sizes_t5):
    print('#### ERROR: wires_coefs_t5 and wires_sizes_t5 HAVE DIFFERENT SIZES')

# Creates de wires declarations
wires_declarations_t5 = []
for w in range(len(wires_coefs_t5)):
    if wires_coefs_t5[w] >= 0:
        wires_declarations_t5.append('  wire signed ['+str(wires_sizes_t5[w]-1)+':0] w'+str(wires_coefs_t5[w])+';')
    else:
        wires_declarations_t5.append('  wire signed ['+str(wires_sizes_t5[w]-1)+':0] w'+str(abs(wires_coefs_t5[w]))+'_;')

wires_declarations_t5_concat = '\n'.join(wires_declarations_t5)

# Creates the outputs sizes for declaration
output_declarations_t5 = []
for i in range(FRAC):
    output_declarations_t5.append('  output signed ['+str(coefs_sizes[i][5]-1)+':0]\t Y'+str(i+1)+';')

output_declarations_t5_concat = '\n'.join(output_declarations_t5)

# print(output_declarations_t5_concat)
# print(wires_declarations_t5_concat)

In [ ]:
t5_affine_verilog = '''/*------------------------------------------------------------------------------
 * File: t5_affine_'''+str(IN_BIT_WIDTH)+'''.v
 * Date generated: 10/02/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: MCM filter for 1/16 precision coefficients - Tap 5
 *------------------------------------------------------------------------------ */

module t5_affine_'''+str(IN_BIT_WIDTH)+''' (
    X,
    Y1,
    Y2,
    Y3,
    Y4,
    Y5,
    Y6,
    Y7,
    Y8,
    Y9,
    Y10,
    Y11,
    Y12,
    Y13,
    Y14,
    Y15
);


// ------------------------------------------
// Port mode declarations
// ------------------------------------------
  input  signed ['''+str(IN_BIT_WIDTH-1)+''':0] X;
'''+output_declarations_t5_concat+'''


// ------------------------------------------
// Wires declarations
// ------------------------------------------
'''+wires_declarations_t5_concat+'''
	 
	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign w1 = X;
  assign w4 = w1 << 2;
  assign w3 = w4 - w1;
  assign w2 = w1 << 1;

  
// ------------------------------------------
// Outputs
// ------------------------------------------
  assign Y1 = w1;
  assign Y2 = w1;
  assign Y3 = w1;
  assign Y4 = w1;
  assign Y5 = w2;
  assign Y6 = w3;
  assign Y7 = w3;
  assign Y8 = w3;
  assign Y9 = w3;
  assign Y10 = w2;
  assign Y11 = w3;
  assign Y12 = w3;
  assign Y13 = w2;
  assign Y14 = w1;
  assign Y15 = w1;

endmodule //t5_affine_'''+str(IN_BIT_WIDTH)+'''

'''
# print(t5_affine_verilog)
if WRITE_ALL_FILTERS_FILE:
    with open('../verilog/all_filters_'+str(IN_BIT_WIDTH)+'/t5_affine_'+str(IN_BIT_WIDTH)+'.v', 'w') as t5_affine_file:
        t5_affine_file.write(t5_affine_verilog)